# MultiProcessing

threading 모듈과 유사한 API를 사용하여 프로세스 생성을 지원하는 패키지입니다.
multiprocessing 패키지는 지역과 원격 동시성을 모두 제공하며 스레드 대신 서브 프로세스를 사용하여 전역 인터프리터 락(Global Interpreter Lock) 을 효과적으로 피합니다. 이것 때문에, multiprocessing 모듈은 프로그래에게 주어진 서버에서 다중 프로세서를 최대한 활용할 수 있게 합니다. 유닉스와 윈도우에서 모두 실행 가능합니다.


A. 전역 인터프리터 락(Global Interpreter Lock)

1) GIL 이란?

여러개의 스레드가 있을때 스레드간의 동기화를 위해 사용되는 기술 중 하나입니다. 
GIL은 전역에 Lock을 두고 이 Lock을 점유해야만 코드를 실행할 수 있도록 제한합니다.
따라서 동시에 하나 이상의 스레드가 실행되지는 않습니다. 

2) GIL의 장점

멀티 쓰레드 프로그램에서 성능이 떨어질 수도 있지만, CPython, Ruby MRI, Lua interpreter 
등 많은 인터프리터 구현체들이 GIL을 사용하고 있습니다. 그 이유는 우선 GIL을 이용한 multi-threads를 구현하는 것이
병렬 multi-threads를 구현하는 것보다 훨씬 쉽다는 것입니다. 
게다가 이런 병렬 multi-threads 구현체들의 문제는 싱글 쓰레드에서 오히려 더 느려진다는 것입니다. 
그래서 CPython이나 Ruby MRI에서는 GIL을 없애려는 많은 시도가 있었지만, 결국 싱글 쓰레드에서의 성능 저하를 극복하지 못하고 GIL로 돌아왔습니다. 
결국, 파이썬의 창시자인 귀도 반 로섬은 CPython에서 GIL을 없애는 대신 싱글 쓰레드에서 성능을 떨어뜨릴 구현은 받아들이지 않겠다고 
선언하기도 했습니다.

https://wiki.python.org/moin/GlobalInterpreterLock#line-31    

In [ ]:
B. Process

process 클래스는 process 객체를 생성합니다.

1) start()
process 생성한 후 start() 함수를 호출하여야 프로세스가 진행됩니다.

2) join()
프로세스가 진행후, join() 함수를 호출하여야 프로세스가 완료됩니다. 
join() 함수를 호출하지 않으면 수동으로 프로세스를 중지시켜야 합니다.

In [1]:
from multiprocessing import Process
import time

def print_func(continent='Asia'):
    print('The name of continent is : ', continent)

if __name__ == "__main__":  
    names = ['America', 'Europe', 'Africa']
    procs = []
    
    for name in names:       
        proc = Process(target=print_func, args=(name,))
        procs.append(proc)
        proc.start()
   
    for proc in procs:
        proc.join()
        print("hello")
        time.sleep(5)

위의 예제는 names 리스트 길이 만큼 process를 생성해서 각 프로세스 마다 print_func
함수를 호출하여 names 리스트의 값을 매개변수로 전달해 주는 것입니다. 

1) args
프로세스에 매개변수를 전달하기 위해서는 args를 사용합니다. 


C. Queue (First-In-First-Out)

Queue 클래스는 타켓 함수가 프로세스에게 데이터를 사용하도록 전달하는데 유용합니다. 

1) put
데이터 입력

2) get
데이터 출력

In [3]:
from multiprocessing import Process, Queue
import os

def read_queue(queue):   
    while True:
        msg = queue.get()         #큐값을 읽어온다.
        print(msg, end=",")
        if (msg == 'DONE'):       #큐값의 마지막(DONE) 에 이르면 읽기를 멈춘다.          
            print('process id:', os.getpid())
            print('\n')
            break

def write_queue(count, queue):   
    for ii in range(0, count):
        queue.put(ii)             # 큐값을 count만큼 생성해서 큐에 넣고
    queue.put('DONE')             # 큐의 마지막에 DONE 이라는 글자를 넣는다.

if __name__=='__main__':
    queue = Queue() 
    for count in [10, 20, 30]:    # 카운터 값 리스트         
        process = Process(target=read_queue, args=((queue),)) # 프로세스 생성하고 args에 queue를 전달  
        
        process.start()        #프로세스 실행      
        write_queue(count, queue)    #queue에 값 저장 
        process.join()         #프로세스가 완료될때까지 기다림
                                

Sending 10 numbers to Queue() took 0.18849492073059082 seconds
Sending 100 numbers to Queue() took 0.16634607315063477 seconds
Sending 1000 numbers to Queue() took 0.1740427017211914 seconds


1) 위의 코드는 큐를 생성해서 카운트 리스트 길이만큼 for 문을 돌면서 프로세스를 생성하고(3개)
각 프로세스에 타켓함수로 read_queue함수를 args로 queue를 전달한다.

2) 각 프로세스를 실행되고 write_queue 함수를 호출되어 리스트의 각 값과 큐를 전달해서
큐에 값을 적재한다. 각 프로세스는 큐에서 Done이라는 글자를 읽을때까지 큐를 읽어들인다.

In [ ]:
아래는 Manager.dict를 이용하는 방법입니다.

In [ ]:
from multiprocessing import Process, Manager

def f(d):
    d[1] += '1'
    d['2'] += 2

if __name__ == '__main__':
    manager = Manager()

    d = manager.dict()
    d[1] = '1'
    d['2'] = 2

    p1 = Process(target=f, args=(d,))
    p2 = Process(target=f, args=(d,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()

    print(d)

D. Lock

락 클래스는 락을 선언할 수 있게 하고 락 해제 전까지 다른 프로세스들이 접근하지 못하게 합니다. 
락 클래스는 락선언과 락해제로 되어 있습니다. 
락 선언시에는 acquire() 함수 사용, 락 해제시에는 release() 함수를 사용합니다.

In [ ]:
from multiprocessing import Process, Lock  

def printer(item, lock): 
    lock.acquire() 
    try: 
        print(item) 
    finally: lock.release()  
        
if __name__ == '__main__': 
    lock = Lock() 
    items = ['tango', 'foxtrot', 10] 
    for item in items: 
        p = Process(target=printer, args=(item, lock)) 
        p.start()
        p.join()

E. Pool

Pool 클래스는 멀티 입력값을 가지고 데이터를 프로세스 사이에 분배시키는 병렬 실행시에 사용가능합니다. 

In [ ]:
from multiprocessing import Pool

import time, os

work = (["A", 5], ["B", 2], ["C", 1], ["D", 3])


def work_log(work_data):
    print(" Process %s waiting %s seconds" % (work_data[0], work_data[1]))
    time.sleep(int(work_data[1]))
    print(" Process %s Finished." % work_data[0])  
    print('process id:', os.getpid())


def pool_handler():
    p = Pool(2)
    p.map(work_log, work)


if __name__ == '__main__':
    pool_handler()

풀 크기는 2 이고 work_log 함수가 병렬로 실행되고 있습니다.
work 리스트의 각 첫번째 값은 키로 두번째값은 sleep 값으로 
두번째값 만큼 work_log함수를 sleep 시킵니다.

풀 크기만큼 병렬로 실행되어 프로세스 아이디가 두개만큼 생성되었음을 알 수 있습니다.